<a href="https://colab.research.google.com/github/ericblanco/AISPACY/blob/main/text_sub_technique_mitre_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install thinc

In [ ]:
!pip install textacy

In [ ]:
!pip install spacytextblob

In [ ]:
!pip install amrlib

In [ ]:
import os
import numpy as np
from spacy.tokens import Doc
from spacy.vocab import Vocab
import spacy 
from spacy import displacy
!python -m spacy download en_core_web_md
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
import matplotlib.pyplot as plt
import sklearn
import keras
import random 
from collections import Counter #for counting
import warnings
import textacy
warnings.filterwarnings('ignore', category=DeprecationWarning)
# warnings.simplefilter('once')

     |████████████████████████████████| 33.5 MB 154 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
from spacy.tokens import Doc

In [ ]:
import pandas as pd

In [ ]:
from collections import defaultdict
from typing import Optional, Iterable, Callable
import thinc.api

In [ ]:
from typing import List, Dict, Union, Any, Optional
from typing import Tuple
import srsly
from pathlib import Path


In [ ]:
nlp = spacy.load('en_core_web_md')


In [ ]:
pipeline = ["tok2vec", "tagger", "textcat", "parser","spacytextblob", "ner"]

In [ ]:
en = spacy.load('en_core_web_md')

all_stopwords = en.Defaults.stop_words

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/lensupdate.csv')

In [ ]:
print('Original DF size:{}'.format(len(df)))

Original DF size:4936


In [ ]:
df.dropna(axis = 0, how ='any')

,id,subtech,sentence
0,T1598.002,Phishing for Information Spearphishing Attachment,Astaroth has been delivered via malicious e-ma...
1,T1598.002,Phishing for Information Spearphishing Attachment,Sidewinder has sent e-mails with malicious att...
2,T1598.002,Phishing for Information Spearphishing Attachment,A phishing campaign has been observed by resea...
3,T1598.002,Phishing for Information Spearphishing Attachment,Researchers have discovered a wave of emails w...
4,T1598.002,Phishing for Information Spearphishing Attachment,Using attached HTML files containing JavaScrip...
...,...,...,...
4931,T1546.001,Event Triggered Execution Change Default File ...,Kimsuky has a HWP document stealer module whic...
4932,T1546.001,Event Triggered Execution Change Default File ...,Cring Ransomware can escalate privileges via c...
4933,T1546.001,Event Triggered Execution Change Default File ...,"Through manipulation of the registry, FIN13 ca..."
4934,T1546.001,Event Triggered Execution Change Default File ...,CobaltStrike contains modules that can modify ...


In [ ]:
df_sentence = str(df['sentence'])

In [ ]:
df_ids = str(df['id'])

In [ ]:
df_subtech = str(df['subtech'])

In [ ]:
doc_sentence = nlp(df_sentence)

In [ ]:
doc_ids = nlp(df_ids)


In [ ]:
doc_subtech = nlp(df_subtech)

In [ ]:
doc_subtech

0       Phishing for Information Spearphishing Attachment
1       Phishing for Information Spearphishing Attachment
2       Phishing for Information Spearphishing Attachment
3       Phishing for Information Spearphishing Attachment
4       Phishing for Information Spearphishing Attachment
                              ...                        
4931    Event Triggered Execution Change Default File ...
4932    Event Triggered Execution Change Default File ...
4933    Event Triggered Execution Change Default File ...
4934    Event Triggered Execution Change Default File ...
4935    Event Triggered Execution Change Default File ...
Name: subtech, Length: 4936, dtype: object

In [ ]:
doc_ids

0       T1598.002
1       T1598.002
2       T1598.002
3       T1598.002
4       T1598.002
          ...    
4931    T1546.001
4932    T1546.001
4933    T1546.001
4934    T1546.001
4935    T1546.001
Name: id, Length: 4936, dtype: object

In [ ]:
for token in doc_subtech:
    print(token.text, token.pos_, token.tag_)

0 X LS
       SPACE _SP
Phishing VERB VBG
for ADP IN
Information PROPN NNP
Spearphishing PROPN NNP
Attachment PROPN NNP

 SPACE _SP
1 NUM CD
       SPACE _SP
Phishing VERB VBG
for ADP IN
Information PROPN NNP
Spearphishing PROPN NNP
Attachment PROPN NNP

 SPACE _SP
2 NUM CD
       SPACE _SP
Phishing VERB VBG
for ADP IN
Information PROPN NNP
Spearphishing PROPN NNP
Attachment PROPN NNP

 SPACE _SP
3 NUM CD
       SPACE _SP
Phishing VERB VBG
for ADP IN
Information PROPN NNP
Spearphishing PROPN NNP
Attachment PROPN NNP

 SPACE _SP
4 NUM CD
       SPACE _SP
Phishing VERB VBG
for ADP IN
Information PROPN NNP
Spearphishing PROPN NNP
Attachment PROPN NNP

                               SPACE _SP
... PUNCT :
                       
 SPACE _SP
4931 NUM CD
    SPACE _SP
Event PROPN NNP
Triggered PROPN NNP
Execution PROPN NNP
Change PROPN NNP
Default PROPN NNP
File PROPN NNP
... PUNCT :

 SPACE _SP
4932 NUM CD
    SPACE _SP
Event PROPN NNP
Triggered PROPN NNP
Execution PROPN NNP
Change PROPN NNP


# Computational Linguistics

Using the doc object to create our corpus for the machine learning algorithms. When creating a corpus for `gensim/scikit-learn`, we sometimes forget the incredible power which `spaCy` packs in its pipeline, so we will briefly demonstrate the same in this section with a smaller example sentence.


In [ ]:
sent = nlp('A phishing campaign has been observed by researchers from Trend Micro that contain a macro-enabled document that exploits the legitimate script engine')

## 🔖 POS-Tagging

In [ ]:
for token in sent:
    print(token.text, token.pos_, token.tag_)

A DET DT
phishing VERB VBG
campaign NOUN NN
has AUX VBZ
been AUX VBN
observed VERB VBN
by ADP IN
researchers NOUN NNS
from ADP IN
Trend PROPN NNP
Micro PROPN NNP
that PRON WDT
contain VERB VBP
a DET DT
macro ADJ JJ
- ADJ JJ
enabled ADJ JJ
document NOUN NN
that PRON WDT
exploits VERB VBZ
the DET DT
legitimate ADJ JJ
script NOUN NN
engine NOUN NN


## 🔖 NER-Tagging

In [ ]:
for token in sent:
    print(token.text, token.ent_type_, token.shape_, token.is_alpha, token.is_stop)

A  X True True
phishing  xxxx True False
campaign  xxxx True False
has  xxx True True
been  xxxx True True
observed  xxxx True False
by  xx True True
researchers  xxxx True False
from  xxxx True True
Trend ORG Xxxxx True False
Micro ORG Xxxxx True False
that  xxxx True True
contain  xxxx True False
a  x True True
macro  xxxx True False
-  - False False
enabled  xxxx True False
document  xxxx True False
that  xxxx True True
exploits  xxxx True False
the  xxx True True
legitimate  xxxx True False
script  xxxx True False
engine  xxxx True False


# New Section

In [ ]:
for ent in sent.ents:
    #print(ent.text, ent.label_)
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

AttributeError: ignored

In [ ]:
displacy.render(sent, style='ent', jupyter=True)

## Dependency Parsing
Dependency parsing is the process of analyzing the grammatical structure of a sentence based on the dependencies between the words in a sentence.

In [ ]:
for chunk in doc_sentence.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

      Astaroth Astaroth nsubjpass delivered
malicious e-ma ma pobj via

1       Sidewinder Sidewinder nsubj sent
e e dobj sent
- - dobj sent
mails mails dobj sent
malicious att att pobj with
A phishing campaign campaign nsubjpass observed
resea resea pobj by
Researchers Researchers nsubj discovered
a wave wave dobj discovered
emails emails pobj of
Using Using nsubj attached
JavaScrip JavaScrip dobj containing
                       
4931    Kimsuky Kimsuky nsubj has
a HWP document stealer module whic whic nsubj escalate
Cring Ransomware Ransomware pobj    
privileges privileges dobj escalate
c c pobj via
manipulation manipulation pobj Through
the registry registry pobj of

4934    CobaltStrike CobaltStrike nsubj contains
modules modules dobj contains
that that nsubj modify
   Neurevt trojan trojan nsubj persists
itself itself dobj persists
a...
Name: sentence sentence dobj executing
Length Length appos sentence


In [ ]:
for email in doc_sentence:
  print(token.text, token.dep_, token.head.text, token.head.pos_, token.tag_, token.shape_, token.is_alpha, token.is_stop,
        [child for child in token.children])

object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN xxxx True False []
object advcl discovered VERB NN

In [ ]:
displacy.render(doc_sentence, style='dep', jupyter=True, options={'distance':80})

# Cleaning
look at the output of the doc object. It seems like nothing, right? But spaCy's internal  structure has done all the work

In [ ]:
nlp(doc)

This is a test of the spaCy extension. The test has multiple sentences.

In [ ]:
from spacy.tokens import DocBin
from spacy.cli.train import train
# the DocBin will store the example documents
docs = df_sentence
for docs_ids_, annotations in training_data:
doc_bin = DocBin(attrs=["ENT_IOB", "ENT_TYPE"])
for start, end, label in annotations:
  span = doc.char_span(start, end, label=label)
  ents.append(span)
  doc.ents = ents
  db.add(doc)
  db.to_disk("/content/drive/MyDrive/trainingdata/train.spacy")

IndentationError: ignored

In [ ]:
df_sentence = str(df['sentence'])

In [ ]:
!pip show amrlib

Name: amrlib
Version: 0.7.1
Summary: A python library that makes AMR parsing, generation and visualization simple.
Home-page: https://github.com/bjascob/amrlib
Author: Brad Jascob
Author-email: bjascob@msn.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: penman, torch, numpy, tqdm, transformers, spacy, smatch
Required-by: 


In [ ]:
graphs = doc._.to_amr()
for graph in graphs:
    print(graph)

FileNotFoundError: ignored

In [ ]:
import amrlib
amrlib.setup_spacy_extension()
# nlp = spacy.load('en_core_web_sm')
doc = nlp('This is a test of the spaCy extension. The test has multiple sentences.')


In [ ]:
graphs = doc._.to_amr()
for graph in graphs:
    print(graph)

FileNotFoundError: ignored

> - Magic of spaCy - just like that, we've managed to get rid of stopwords, puctuation markers, and added lemmatized word.
> - Sometimes topic modeling make more sense when `Magic` and `mountain` are treated as `Magic Mountain` - By creating a bigram model and modifying our corpus accordingly.